# Data Processing and Modeling for Zillow's Home Value Prediction

The train data has all the transactions before October 15, 2016, plus some of the transactions after October 15, 2016

The test data in the public leaderboard has the rest of the transactions between October 15 and December 31, 2016.

You are asked to predict 6 time points for all properties: October 2016 (201610), November 2016 (201611), December 2016 (201612), October 2017 (201710), November 2017 (201711), and December 2017 (201712).

Not all the properties are sold in each time period. If a property was not sold in a certain time period, that particular row will be ignored when calculating your score.

In [1]:
import src.data_proc as data_proc

import numpy as np
import pandas as pd
import sys
import os
import gc
import random
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None
pd.options.display.float_format

from sklearn.model_selection import train_test_split

import lightgbm as lgb

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
%%time
# Load in properties data
prop_2016 = data_proc.load_properties_data("data/properties_2016.csv")
prop_2017 = data_proc.load_properties_data("data/properties_2017.csv")

assert len(prop_2016) == len(prop_2017)
print("Number of properties: {}".format(len(prop_2016)))
print("Number of property features: {}".format(len(prop_2016.columns)-1))

Number of properties: 2985217
Number of property features: 57
CPU times: user 34.2 s, sys: 6.7 s, total: 40.9 s
Wall time: 41 s


In [3]:
# Load in training data (with logerror labels)
train_2016 = data_proc.load_training_data("data/train_2016_v2.csv")
train_2017 = data_proc.load_training_data("data/train_2017.csv")

print("Number of 2016 transaction records: {}".format(len(train_2016)))
print("Number of 2016 transaction records: {}".format(len(train_2017)))
print("\n", train_2016.head())
print("\n", train_2017.head())

Number of 2016 transaction records: 90275
Number of 2016 transaction records: 77613

    parcelid  logerror transactiondate
0  11016594    0.0276      2016-01-01
1  14366692   -0.1684      2016-01-01
2  12098116   -0.0040      2016-01-01
3  12643413    0.0218      2016-01-02
4  14432541   -0.0050      2016-01-02

    parcelid  logerror transactiondate
0  14297519  0.025595      2017-01-01
1  17052889  0.055619      2017-01-01
2  14186244  0.005383      2017-01-01
3  12177905 -0.103410      2017-01-01
4  10887214  0.006940      2017-01-01


In [4]:
# Rename & retype the feature columns; also unify representations of missing values
def get_landuse_code_df(prop_2016, prop_2017):
    temp = prop_2016.groupby('county_landuse_code')['county_landuse_code'].count()
    landuse_codes = list(temp[temp >= 300].index)
    temp = prop_2017.groupby('county_landuse_code')['county_landuse_code'].count()
    landuse_codes += list(temp[temp >= 300].index)
    landuse_codes = list(set(landuse_codes))
    df_landuse_codes = pd.DataFrame({'county_landuse_code': landuse_codes,
                                     'county_landuse_code_id': range(len(landuse_codes))})
    return df_landuse_codes

def get_zoning_desc_code_df(prop_2016, prop_2017):
    temp = prop_2016.groupby('zoning_description')['zoning_description'].count()
    zoning_codes = list(temp[temp >= 5000].index)
    temp = prop_2017.groupby('zoning_description')['zoning_description'].count()
    zoning_codes += list(temp[temp >= 5000].index)
    zoning_codes = list(set(zoning_codes))
    df_zoning_codes = pd.DataFrame({'zoning_description': zoning_codes,
                                     'zoning_description_id': range(len(zoning_codes))})
    return df_zoning_codes

def process_columns(df, df_landuse_codes, df_zoning_codes):
    df = df.merge(how='left', right=df_landuse_codes, on='county_landuse_code')
    df = df.drop(['county_landuse_code'], axis=1)
    
    df = df.merge(how='left', right=df_zoning_codes, on='zoning_description')
    df = df.drop(['zoning_description'], axis=1)
    
    df.loc[df.county_id == 3101, 'county_id'] = 0
    df.loc[df.county_id == 1286, 'county_id'] = 1
    df.loc[df.county_id == 2061, 'county_id'] = 2
    return df

data_proc.rename_columns(prop_2016)
data_proc.rename_columns(prop_2017)

df_landuse_codes = get_landuse_code_df(prop_2016, prop_2017)
df_zoning_codes = get_zoning_desc_code_df(prop_2016, prop_2017)
prop_2016 = process_columns(prop_2016, df_landuse_codes, df_zoning_codes)
prop_2017 = process_columns(prop_2017, df_landuse_codes, df_zoning_codes)

data_proc.retype_columns(prop_2016)
data_proc.retype_columns(prop_2017)

prop_2017.head()

,parcelid,cooling_id,architecture_style_id,basement_sqft,bathroom_cnt,bedroom_cnt,framing_id,quality_id,bathroom_cnt_calc,deck_id,floor1_sqft,finished_area_sqft_calc,finished_area_sqft,perimeter_area,total_area,floor1_sqft_unk,base_total_area,fips,fireplace_cnt,bathroom_full_cnt,garage_cnt,garage_sqft,spa_flag,heating_id,latitude,longitude,lot_sqft,pool_cnt,pool_total_size,pool_unk_1,pool_unk_2,pool_unk_3,landuse_type_id,census_1,city_id,county_id,neighborhood_id,region_zip,room_cnt,story_id,bathroom_small_cnt,construction_id,unit_cnt,patio_sqft,storage_sqft,year_built,story_cnt,fireplace_flag,tax_structure,tax_parcel,tax_year,tax_land,tax_property,tax_overdue_flag,tax_overdue_year,census_2,county_landuse_code_id,zoning_description_id
0,10754147,-1,-1,NaN,0.0,0.0,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34144440.0,-118654080.0,85768.0,NaN,NaN,NaN,NaN,NaN,238,60378004.0,37688.0,0,NaN,96337.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN,1,-1
1,10759547,-1,-1,NaN,0.0,0.0,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34140432.0,-118625360.0,4083.0,NaN,NaN,NaN,NaN,NaN,230,60378000.0,37688.0,0,NaN,96337.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,8,28
2,10843547,-1,-1,NaN,0.0,0.0,4,-1,NaN,NaN,NaN,73026.0,NaN,NaN,73026.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,33989360.0,-118394632.0,63085.0,NaN,NaN,NaN,NaN,NaN,16,60377032.0,51617.0,0,NaN,96095.0,0.0,NaN,NaN,-1,2.0,NaN,NaN,1959.0,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.369141,NaN,NaN,NaN,25,51
3,10859147,-1,-1,NaN,0.0,0.0,2,5,NaN,NaN,NaN,5068.0,NaN,NaN,5068.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34148864.0,-118437208.0,7521.0,NaN,NaN,NaN,NaN,NaN,16,60371412.0,12447.0,0,27080.0,96424.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,1948.0,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.570312,NaN,NaN,NaN,25,51
4,10879947,-1,-1,NaN,0.0,0.0,3,-1,NaN,NaN,NaN,1776.0,NaN,NaN,1776.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34194168.0,-118385816.0,8512.0,NaN,NaN,NaN,NaN,NaN,0,60371232.0,12447.0,0,46795.0,96450.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1947.0,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.169922,NaN,NaN,NaN,14,-1


In [5]:
# Join the training data with the property table
train_2016 = train_2016.merge(how='left', right=prop_2016, on='parcelid')
train_2017 = train_2017.merge(how='left', right=prop_2017, on='parcelid')

# Remove outliers from training set
threshold = 0.4
print("Remove outliers for 2016 training set:")
train_2016 = data_proc.remove_outliers(train_2016, threshold)
print("\nRemove outliers for 2017 training set:")
train_2017 = data_proc.remove_outliers(train_2017, threshold)

# Combine the 2016 and 2017 training sets
train = pd.concat([train_2016, train_2017], axis=0, ignore_index=True)
print("\nCombined training set size: {}".format(len(train)))

train.head(20)

Remove outliers for 2016 training set:
90275 training examples in total
1844 with abs(logerror) > 0.4
New training set size (outliers removed): 88431

Remove outliers for 2017 training set:
77613 training examples in total
1745 with abs(logerror) > 0.4
New training set size (outliers removed): 75868

Combined training set size: 164299


,parcelid,logerror,transactiondate,cooling_id,architecture_style_id,basement_sqft,bathroom_cnt,bedroom_cnt,framing_id,quality_id,bathroom_cnt_calc,deck_id,floor1_sqft,finished_area_sqft_calc,finished_area_sqft,perimeter_area,total_area,floor1_sqft_unk,base_total_area,fips,fireplace_cnt,bathroom_full_cnt,garage_cnt,garage_sqft,spa_flag,heating_id,latitude,longitude,lot_sqft,pool_cnt,pool_total_size,pool_unk_1,pool_unk_2,pool_unk_3,landuse_type_id,census_1,city_id,county_id,neighborhood_id,region_zip,room_cnt,story_id,bathroom_small_cnt,construction_id,unit_cnt,patio_sqft,storage_sqft,year_built,story_cnt,fireplace_flag,tax_structure,tax_parcel,tax_year,tax_land,tax_property,tax_overdue_flag,tax_overdue_year,census_2,county_landuse_code_id,zoning_description_id
0,11016594,0.0276,2016-01-01,0,-1,NaN,2.0,3.0,-1,3,2.0,NaN,NaN,1684.0,1684.0,NaN,NaN,NaN,NaN,0,NaN,2.0,NaN,NaN,NaN,1,34280992.0,-118488536.0,7528.0,NaN,NaN,NaN,NaN,NaN,230,60371068.0,12447.0,0,31817.0,96370.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1959.0,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.879883,NaN,NaN,6.037107e+13,29,29
1,14366692,-0.1684,2016-01-01,-1,-1,NaN,3.5,4.0,-1,-1,3.5,NaN,NaN,2263.0,2263.0,NaN,NaN,NaN,NaN,22,NaN,3.0,2.0,468.0,NaN,-1,33668120.0,-117677552.0,3643.0,NaN,NaN,NaN,NaN,NaN,230,60590524.0,32380.0,1,NaN,96962.0,0.0,NaN,1.0,-1,NaN,NaN,NaN,2014.0,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.019531,NaN,NaN,NaN,22,-1
2,12098116,-0.0040,2016-01-01,0,-1,NaN,3.0,2.0,-1,3,3.0,NaN,NaN,2217.0,2217.0,NaN,NaN,NaN,NaN,0,NaN,3.0,NaN,NaN,NaN,1,34136312.0,-118175032.0,11423.0,NaN,NaN,NaN,NaN,NaN,230,60374640.0,47019.0,0,275411.0,96293.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1940.0,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.480469,NaN,NaN,6.037464e+13,29,32
3,12643413,0.0218,2016-01-02,0,-1,NaN,2.0,2.0,-1,3,2.0,NaN,NaN,839.0,839.0,NaN,NaN,NaN,NaN,0,NaN,2.0,NaN,NaN,NaN,1,33755800.0,-118309000.0,70859.0,NaN,NaN,NaN,NaN,NaN,235,60372964.0,12447.0,0,54300.0,96222.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1987.0,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.739990,NaN,NaN,6.037296e+13,37,50
4,14432541,-0.0050,2016-01-02,-1,-1,NaN,2.5,4.0,-1,-1,2.5,NaN,NaN,2283.0,2283.0,NaN,NaN,NaN,NaN,22,NaN,2.0,2.0,598.0,NaN,-1,33485644.0,-117700232.0,6000.0,1.0,NaN,NaN,NaN,1.0,230,60590424.0,17686.0,1,NaN,96961.0,8.0,NaN,1.0,-1,NaN,NaN,NaN,1981.0,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.959961,NaN,NaN,6.059042e+13,50,-1
5,11509835,-0.2705,2016-01-02,0,-1,NaN,4.0,4.0,-1,0,4.0,NaN,NaN,3067.0,3067.0,NaN,NaN,NaN,NaN,0,NaN,4.0,NaN,NaN,NaN,1,33870088.0,-118402768.0,2708.0,NaN,NaN,NaN,NaN,NaN,230,60376212.0,29712.0,0,NaN,96109.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1982.0,NaN,NaN,880650.0,2447951.0,2015.0,1567301.0,27126.570312,NaN,NaN,6.037621e+13,29,-1
6,12286022,0.0440,2016-01-02,-1,-1,NaN,1.0,2.0,-1,6,1.0,NaN,NaN,1297.0,1297.0,NaN,NaN,NaN,NaN,0,NaN,1.0,NaN,NaN,NaN,6,33899476.0,-118212720.0,6677.0,NaN,NaN,NaN,NaN,NaN,230,60375416.0,24174.0,0,NaN,96091.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1939.0,NaN,NaN,64549.0,111521.0,2015.0,46972.0,2304.969971,NaN,NaN,6.037542e+13,29,-1
7,17177301,0.1638,2016-01-02,-1,-1,NaN,2.5,3.0,-1,-1,2.5,NaN,853.0,1763.0,1763.0,NaN,NaN,853.0,NaN,74,1.0,2.0,2.0,0.0,NaN,-1,34207204.0,-119165592.0,NaN,NaN,NaN,NaN,NaN,NaN,235,61110032.0,13150.0,2,NaN,97101.0,6.0,NaN,1.0,-1,NaN,NaN,NaN,1994.0,2.0,NaN,107000.0,306000.0,2015.0,199000.0,3745.500000,NaN,NaN,6.111003e+13,41,-1
8,14739064,-0.0030,2016-01-02,-1,-1,NaN,1.0,2.0,-1,-1,1.0,NaN,NaN,796.0,796.0,NaN,NaN,NaN,NaN,22,NaN,1.0,1.0,0.0,NaN,-1,33549600.0,-117678000.0,NaN,NaN,NaN,NaN,NaN,NaN,235,60590424.0,25459.0,1,NaN,96987.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,1984.0,NaN,NaN,66834.0,210064.0,2015.0,143230.0,2172.879883,NaN,NaN,6.059042e+13,35,-1
9,14677559,0.0843,2016-01-03,-1,-1,NaN,2.0,2.0,-1,-1,2.0,NaN,NaN,1260.0,1260.0,NaN,NaN,NaN,NaN,22,NaN,2.0,1.0,0.0,NaN,-1,33612700.0,-117742000.0,NaN,NaN,NaN,NaN,NaN,NaN,235,60590628.0,46098.0,1,NaN,96963.0,5.0,NaN,NaN,-1,NaN,NaN,NaN,1977.0,1.0,NaN,109977.0,190960.0,2015.0,80983.0,1940.260010,NaN,NaN,6.059063e+13,35,-1


In [6]:
# Add datetime features
logerror_year, logerror_month, logerror_quarter = data_proc.compute_datetime_aggregate_features(train)
train = data_proc.add_datetime_aggregate_features(train, logerror_year, logerror_month, logerror_quarter)    

# TODO: Derive location related features

# Drop features that are not useful or too messy
lgb_features = data_proc.drop_features(train)

print("Number of features for LightGBM: {}".format(len(lgb_features.columns)))
lgb_features.head(10)

Number of features for LightGBM: 54


,cooling_id,bathroom_cnt,bedroom_cnt,quality_id,bathroom_cnt_calc,deck_id,floor1_sqft,finished_area_sqft_calc,finished_area_sqft,total_area,floor1_sqft_unk,base_total_area,fips,fireplace_cnt,bathroom_full_cnt,garage_cnt,garage_sqft,spa_flag,heating_id,latitude,longitude,lot_sqft,pool_cnt,pool_total_size,pool_unk_1,pool_unk_2,pool_unk_3,landuse_type_id,census_1,city_id,county_id,neighborhood_id,region_zip,room_cnt,bathroom_small_cnt,construction_id,unit_cnt,patio_sqft,year_built,story_cnt,fireplace_flag,tax_structure,tax_parcel,tax_year,tax_land,tax_property,tax_overdue_flag,tax_overdue_year,census_2,county_landuse_code_id,zoning_description_id,logerror_year,logerror_month,logerror_quarter
0,0,2.0,3.0,3,2.0,NaN,NaN,1684.0,1684.0,NaN,NaN,NaN,0,NaN,2.0,NaN,NaN,NaN,1,34280992.0,-118488536.0,7528.0,NaN,NaN,NaN,NaN,NaN,230,60371068.0,12447.0,0,31817.0,96370.0,0.0,NaN,-1,1.0,NaN,1959.0,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.879883,NaN,NaN,6.037107e+13,29,29,0.005,0.01,0.006
1,-1,3.5,4.0,-1,3.5,NaN,NaN,2263.0,2263.0,NaN,NaN,NaN,22,NaN,3.0,2.0,468.0,NaN,-1,33668120.0,-117677552.0,3643.0,NaN,NaN,NaN,NaN,NaN,230,60590524.0,32380.0,1,NaN,96962.0,0.0,1.0,-1,NaN,NaN,2014.0,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.019531,NaN,NaN,NaN,22,-1,0.005,0.01,0.006
2,0,3.0,2.0,3,3.0,NaN,NaN,2217.0,2217.0,NaN,NaN,NaN,0,NaN,3.0,NaN,NaN,NaN,1,34136312.0,-118175032.0,11423.0,NaN,NaN,NaN,NaN,NaN,230,60374640.0,47019.0,0,275411.0,96293.0,0.0,NaN,-1,1.0,NaN,1940.0,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.480469,NaN,NaN,6.037464e+13,29,32,0.005,0.01,0.006
3,0,2.0,2.0,3,2.0,NaN,NaN,839.0,839.0,NaN,NaN,NaN,0,NaN,2.0,NaN,NaN,NaN,1,33755800.0,-118309000.0,70859.0,NaN,NaN,NaN,NaN,NaN,235,60372964.0,12447.0,0,54300.0,96222.0,0.0,NaN,-1,1.0,NaN,1987.0,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.739990,NaN,NaN,6.037296e+13,37,50,0.005,0.01,0.006
4,-1,2.5,4.0,-1,2.5,NaN,NaN,2283.0,2283.0,NaN,NaN,NaN,22,NaN,2.0,2.0,598.0,NaN,-1,33485644.0,-117700232.0,6000.0,1.0,NaN,NaN,NaN,1.0,230,60590424.0,17686.0,1,NaN,96961.0,8.0,1.0,-1,NaN,NaN,1981.0,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.959961,NaN,NaN,6.059042e+13,50,-1,0.005,0.01,0.006
5,0,4.0,4.0,0,4.0,NaN,NaN,3067.0,3067.0,NaN,NaN,NaN,0,NaN,4.0,NaN,NaN,NaN,1,33870088.0,-118402768.0,2708.0,NaN,NaN,NaN,NaN,NaN,230,60376212.0,29712.0,0,NaN,96109.0,0.0,NaN,-1,1.0,NaN,1982.0,NaN,NaN,880650.0,2447951.0,2015.0,1567301.0,27126.570312,NaN,NaN,6.037621e+13,29,-1,0.005,0.01,0.006
6,-1,1.0,2.0,6,1.0,NaN,NaN,1297.0,1297.0,NaN,NaN,NaN,0,NaN,1.0,NaN,NaN,NaN,6,33899476.0,-118212720.0,6677.0,NaN,NaN,NaN,NaN,NaN,230,60375416.0,24174.0,0,NaN,96091.0,0.0,NaN,-1,1.0,NaN,1939.0,NaN,NaN,64549.0,111521.0,2015.0,46972.0,2304.969971,NaN,NaN,6.037542e+13,29,-1,0.005,0.01,0.006
7,-1,2.5,3.0,-1,2.5,NaN,853.0,1763.0,1763.0,NaN,853.0,NaN,74,1.0,2.0,2.0,0.0,NaN,-1,34207204.0,-119165592.0,NaN,NaN,NaN,NaN,NaN,NaN,235,61110032.0,13150.0,2,NaN,97101.0,6.0,1.0,-1,NaN,NaN,1994.0,2.0,NaN,107000.0,306000.0,2015.0,199000.0,3745.500000,NaN,NaN,6.111003e+13,41,-1,0.005,0.01,0.006
8,-1,1.0,2.0,-1,1.0,NaN,NaN,796.0,796.0,NaN,NaN,NaN,22,NaN,1.0,1.0,0.0,NaN,-1,33549600.0,-117678000.0,NaN,NaN,NaN,NaN,NaN,NaN,235,60590424.0,25459.0,1,NaN,96987.0,0.0,NaN,-1,NaN,NaN,1984.0,NaN,NaN,66834.0,210064.0,2015.0,143230.0,2172.879883,NaN,NaN,6.059042e+13,35,-1,0.005,0.01,0.006
9,-1,2.0,2.0,-1,2.0,NaN,NaN,1260.0,1260.0,NaN,NaN,NaN,22,NaN,2.0,1.0,0.0,NaN,-1,33612700.0,-117742000.0,NaN,NaN,NaN,NaN,NaN,NaN,235,60590628.0,46098.0,1,NaN,96963.0,5.0,NaN,-1,NaN,NaN,1977.0,1.0,NaN,109977.0,190960.0,2015.0,80983.0,1940.260010,NaN,NaN,6.059063e+13,35,-1,0.005,0.01,0.006


In [7]:
# Prepare data for LightGBM
lgb_label = train.logerror.astype(np.float32)
print(lgb_label.head())

# Transform to Numpy matrices and perform train/val split
lgb_X = lgb_features.values
lgb_y = lgb_label.values

np.random.seed(42)
random.seed(10)
X_train, X_val, y_train, y_val = train_test_split(lgb_X, lgb_y, test_size=0.2)
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_val shape: {}".format(X_val.shape))
print("y_val shape: {}".format(y_val.shape))

0    0.0276
1   -0.1684
2   -0.0040
3    0.0218
4   -0.0050
Name: logerror, dtype: float32
X_train shape: (131439, 54)
y_train shape: (131439,)
X_val shape: (32860, 54)
y_val shape: (32860,)


In [8]:
# LightGBM parameters
params = {}

params['objective'] = 'regression'
params['metric'] = 'mae'
params['num_threads'] = 4  # set to number of real CPU cores for best performance

params['boosting_type'] = 'gbdt'
params['num_boost_round'] = 2000
params['learning_rate'] = 0.001  # shrinkage_rate
params['early_stopping_rounds'] = 20  # Early stopping based on validation set performance

# Control tree growing
params['num_leaves'] = 512  # max number of leaves in one tree (default 31)
params['min_data'] = 100  # min_data_in_leaf
params['min_hessian'] = 0.05  # min_sum_hessian_in_leaf (default 1e-3)
params['max_depth'] = -1  # limit the max depth of tree model, defult -1 (no limit)
params['max_bin'] = 255  # max number of bins that feature values are bucketed in (small -> less overfitting, default 255)
params['sub_feature'] = 0.65    # feature_fraction (small values => use very different submodels)

# Row subsampling (speed up training and alleviate overfitting)
params['bagging_fraction'] = 0.75
params['bagging_freq'] = 50  # perform bagging at every k iteration

# Constraints on categorical features
params['min_data_per_group'] = 100  # minimal number of data per categorical group (default 100)
params['cat_smooth'] = 15.0  # reduce effect of noises in categorical features, especially for those with few data (default 10.0)

# Regularization (default 0.0)
params['lambda_l1'] = 0.0
params['lambda_l2'] = 0.0

# Random seeds (keep default values)
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3

In [9]:
# Train LightGBM
feature_names = [s for s in lgb_features.columns]
categorical_indices = []
for i, t in enumerate(lgb_features.dtypes):
    if t.name == 'category':
        categorical_indices.append(i)

lgb_train_set = lgb.Dataset(X_train, label=y_train, feature_name=feature_names)
lgb_valid_set = lgb.Dataset(X_val, label=y_val, feature_name=feature_names)

np.random.seed(42)
random.seed(36)
clf = lgb.train(params, lgb_train_set, verbose_eval=True,
                valid_sets=[lgb_train_set, lgb_valid_set], valid_names=['train', 'val'],
                categorical_feature=categorical_indices)

# Evaluate on validation set
print("Train score: {}".format(abs(clf.predict(X_train) - y_train).mean() * 100))
print("Val score: {}".format(abs(clf.predict(X_val) - y_val).mean() * 100))

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1190: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 3, 12, 18, 30, 35, 49, 50]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	train's l1: 0.0527783	val's l1: 0.052211
Training until validation scores don't improve for 20 rounds.
[2]	train's l1: 0.0527753	val's l1: 0.0522094
[3]	train's l1: 0.0527728	val's l1: 0.0522082
[4]	train's l1: 0.0527701	val's l1: 0.0522066
[5]	train's l1: 0.0527677	val's l1: 0.0522052
[6]	train's l1: 0.0527648	val's l1: 0.0522038
[7]	train's l1: 0.0527621	val's l1: 0.0522024
[8]	train's l1: 0.0527591	val's l1: 0.0522008
[9]	train's l1: 0.0527561	val's l1: 0.052199
[10]	train's l1: 0.0527531	val's l1: 0.0521974
[11]	train's l1: 0.0527501	val's l1: 0.0521957
[12]	train's l1: 0.0527474	val's l1: 0.0521942
[13]	train's l1: 0.0527447	val's l1: 0.0521928
[14]	train's l1: 0.0527419	val's l1: 0.0521912
[15]	train's l1: 0.0527389	val's l1: 0.0521896
[16]	train's l1: 0.0527361	val's l1: 0.0521881
[17]	train's l1: 0.0527331	val's l1: 0.0521864
[18]	train's l1: 0.0527301	val's l1: 0.0521851
[19]	train's l1: 0.0527273	val's l1: 0.0521836
[20]	train's l1: 0.0527246	val's l1: 0.0521823
[21]	trai

[178]	train's l1: 0.0523084	val's l1: 0.0519815
[179]	train's l1: 0.0523062	val's l1: 0.0519804
[180]	train's l1: 0.052304	val's l1: 0.0519796
[181]	train's l1: 0.0523017	val's l1: 0.0519787
[182]	train's l1: 0.0522994	val's l1: 0.0519779
[183]	train's l1: 0.052297	val's l1: 0.0519768
[184]	train's l1: 0.0522944	val's l1: 0.0519758
[185]	train's l1: 0.0522919	val's l1: 0.0519747
[186]	train's l1: 0.0522893	val's l1: 0.0519735
[187]	train's l1: 0.0522868	val's l1: 0.0519723
[188]	train's l1: 0.0522846	val's l1: 0.0519714
[189]	train's l1: 0.0522823	val's l1: 0.0519705
[190]	train's l1: 0.0522801	val's l1: 0.0519694
[191]	train's l1: 0.0522776	val's l1: 0.0519684
[192]	train's l1: 0.0522752	val's l1: 0.0519671
[193]	train's l1: 0.0522726	val's l1: 0.051966
[194]	train's l1: 0.05227	val's l1: 0.0519651
[195]	train's l1: 0.0522675	val's l1: 0.051964
[196]	train's l1: 0.0522651	val's l1: 0.0519628
[197]	train's l1: 0.0522628	val's l1: 0.0519619
[198]	train's l1: 0.0522604	val's l1: 0.051960

[351]	train's l1: 0.0519186	val's l1: 0.0518199
[352]	train's l1: 0.0519165	val's l1: 0.0518191
[353]	train's l1: 0.0519143	val's l1: 0.0518182
[354]	train's l1: 0.0519121	val's l1: 0.0518173
[355]	train's l1: 0.0519101	val's l1: 0.0518166
[356]	train's l1: 0.0519081	val's l1: 0.0518159
[357]	train's l1: 0.0519061	val's l1: 0.0518152
[358]	train's l1: 0.0519041	val's l1: 0.0518145
[359]	train's l1: 0.0519018	val's l1: 0.0518137
[360]	train's l1: 0.0518997	val's l1: 0.0518129
[361]	train's l1: 0.0518976	val's l1: 0.051812
[362]	train's l1: 0.0518956	val's l1: 0.0518112
[363]	train's l1: 0.0518933	val's l1: 0.0518105
[364]	train's l1: 0.0518912	val's l1: 0.0518099
[365]	train's l1: 0.0518892	val's l1: 0.0518092
[366]	train's l1: 0.0518872	val's l1: 0.0518086
[367]	train's l1: 0.0518852	val's l1: 0.0518078
[368]	train's l1: 0.0518833	val's l1: 0.0518071
[369]	train's l1: 0.051881	val's l1: 0.0518064
[370]	train's l1: 0.051879	val's l1: 0.0518058
[371]	train's l1: 0.0518768	val's l1: 0.051

[529]	train's l1: 0.0515661	val's l1: 0.0517043
[530]	train's l1: 0.0515642	val's l1: 0.0517038
[531]	train's l1: 0.0515623	val's l1: 0.0517032
[532]	train's l1: 0.0515604	val's l1: 0.0517026
[533]	train's l1: 0.0515585	val's l1: 0.0517021
[534]	train's l1: 0.0515568	val's l1: 0.0517016
[535]	train's l1: 0.0515549	val's l1: 0.0517014
[536]	train's l1: 0.0515531	val's l1: 0.0517009
[537]	train's l1: 0.0515511	val's l1: 0.0517003
[538]	train's l1: 0.0515493	val's l1: 0.0516998
[539]	train's l1: 0.0515474	val's l1: 0.0516993
[540]	train's l1: 0.0515455	val's l1: 0.0516987
[541]	train's l1: 0.0515436	val's l1: 0.0516984
[542]	train's l1: 0.0515417	val's l1: 0.0516979
[543]	train's l1: 0.05154	val's l1: 0.0516973
[544]	train's l1: 0.0515382	val's l1: 0.0516968
[545]	train's l1: 0.0515363	val's l1: 0.0516962
[546]	train's l1: 0.0515344	val's l1: 0.0516956
[547]	train's l1: 0.0515326	val's l1: 0.0516952
[548]	train's l1: 0.0515308	val's l1: 0.0516948
[549]	train's l1: 0.0515289	val's l1: 0.05

[703]	train's l1: 0.0512606	val's l1: 0.0516228
[704]	train's l1: 0.0512588	val's l1: 0.0516226
[705]	train's l1: 0.0512571	val's l1: 0.0516223
[706]	train's l1: 0.0512554	val's l1: 0.0516218
[707]	train's l1: 0.0512537	val's l1: 0.0516213
[708]	train's l1: 0.0512521	val's l1: 0.0516208
[709]	train's l1: 0.0512504	val's l1: 0.0516204
[710]	train's l1: 0.0512487	val's l1: 0.0516201
[711]	train's l1: 0.051247	val's l1: 0.0516197
[712]	train's l1: 0.0512454	val's l1: 0.0516194
[713]	train's l1: 0.0512437	val's l1: 0.051619
[714]	train's l1: 0.0512421	val's l1: 0.0516186
[715]	train's l1: 0.0512405	val's l1: 0.0516182
[716]	train's l1: 0.0512387	val's l1: 0.051618
[717]	train's l1: 0.051237	val's l1: 0.0516175
[718]	train's l1: 0.0512355	val's l1: 0.0516172
[719]	train's l1: 0.051234	val's l1: 0.0516169
[720]	train's l1: 0.0512322	val's l1: 0.0516165
[721]	train's l1: 0.0512305	val's l1: 0.0516162
[722]	train's l1: 0.0512289	val's l1: 0.0516158
[723]	train's l1: 0.0512273	val's l1: 0.05161

[875]	train's l1: 0.0509841	val's l1: 0.0515649
[876]	train's l1: 0.0509828	val's l1: 0.0515646
[877]	train's l1: 0.0509813	val's l1: 0.0515644
[878]	train's l1: 0.0509798	val's l1: 0.0515642
[879]	train's l1: 0.0509782	val's l1: 0.051564
[880]	train's l1: 0.0509767	val's l1: 0.0515638
[881]	train's l1: 0.0509753	val's l1: 0.0515635
[882]	train's l1: 0.0509739	val's l1: 0.0515632
[883]	train's l1: 0.0509724	val's l1: 0.051563
[884]	train's l1: 0.050971	val's l1: 0.0515627
[885]	train's l1: 0.0509694	val's l1: 0.0515624
[886]	train's l1: 0.0509678	val's l1: 0.0515621
[887]	train's l1: 0.0509663	val's l1: 0.0515619
[888]	train's l1: 0.0509648	val's l1: 0.0515615
[889]	train's l1: 0.0509632	val's l1: 0.0515613
[890]	train's l1: 0.0509616	val's l1: 0.0515609
[891]	train's l1: 0.0509601	val's l1: 0.0515608
[892]	train's l1: 0.0509586	val's l1: 0.0515605
[893]	train's l1: 0.0509569	val's l1: 0.0515602
[894]	train's l1: 0.0509555	val's l1: 0.0515599
[895]	train's l1: 0.050954	val's l1: 0.0515

[1051]	train's l1: 0.0507274	val's l1: 0.0515271
[1052]	train's l1: 0.050726	val's l1: 0.0515268
[1053]	train's l1: 0.0507245	val's l1: 0.0515266
[1054]	train's l1: 0.0507231	val's l1: 0.0515265
[1055]	train's l1: 0.0507216	val's l1: 0.0515263
[1056]	train's l1: 0.0507201	val's l1: 0.0515261
[1057]	train's l1: 0.0507185	val's l1: 0.0515258
[1058]	train's l1: 0.050717	val's l1: 0.0515255
[1059]	train's l1: 0.0507156	val's l1: 0.0515254
[1060]	train's l1: 0.0507141	val's l1: 0.0515251
[1061]	train's l1: 0.0507126	val's l1: 0.0515249
[1062]	train's l1: 0.0507112	val's l1: 0.0515248
[1063]	train's l1: 0.0507099	val's l1: 0.0515246
[1064]	train's l1: 0.0507084	val's l1: 0.0515243
[1065]	train's l1: 0.0507069	val's l1: 0.0515239
[1066]	train's l1: 0.0507055	val's l1: 0.0515237
[1067]	train's l1: 0.0507041	val's l1: 0.0515235
[1068]	train's l1: 0.0507026	val's l1: 0.0515232
[1069]	train's l1: 0.0507013	val's l1: 0.0515231
[1070]	train's l1: 0.0506998	val's l1: 0.0515229
[1071]	train's l1: 0.0

[1223]	train's l1: 0.0504912	val's l1: 0.0515015
[1224]	train's l1: 0.0504899	val's l1: 0.0515014
[1225]	train's l1: 0.0504886	val's l1: 0.0515012
[1226]	train's l1: 0.0504873	val's l1: 0.051501
[1227]	train's l1: 0.0504861	val's l1: 0.0515008
[1228]	train's l1: 0.0504847	val's l1: 0.0515007
[1229]	train's l1: 0.0504835	val's l1: 0.0515007
[1230]	train's l1: 0.050482	val's l1: 0.0515005
[1231]	train's l1: 0.0504807	val's l1: 0.0515004
[1232]	train's l1: 0.0504795	val's l1: 0.0515003
[1233]	train's l1: 0.0504783	val's l1: 0.0515003
[1234]	train's l1: 0.050477	val's l1: 0.0515001
[1235]	train's l1: 0.0504756	val's l1: 0.0514999
[1236]	train's l1: 0.0504743	val's l1: 0.0514999
[1237]	train's l1: 0.0504731	val's l1: 0.0514999
[1238]	train's l1: 0.0504719	val's l1: 0.0514999
[1239]	train's l1: 0.0504706	val's l1: 0.0514998
[1240]	train's l1: 0.0504692	val's l1: 0.0514999
[1241]	train's l1: 0.0504679	val's l1: 0.0514997
[1242]	train's l1: 0.0504666	val's l1: 0.0514997
[1243]	train's l1: 0.05

[1393]	train's l1: 0.0502718	val's l1: 0.0514888
[1394]	train's l1: 0.0502706	val's l1: 0.0514886
[1395]	train's l1: 0.0502693	val's l1: 0.0514886
[1396]	train's l1: 0.0502682	val's l1: 0.0514886
[1397]	train's l1: 0.0502669	val's l1: 0.0514884
[1398]	train's l1: 0.0502656	val's l1: 0.0514883
[1399]	train's l1: 0.0502643	val's l1: 0.0514883
[1400]	train's l1: 0.050263	val's l1: 0.0514884
[1401]	train's l1: 0.0502618	val's l1: 0.0514884
[1402]	train's l1: 0.0502604	val's l1: 0.0514882
[1403]	train's l1: 0.0502591	val's l1: 0.0514883
[1404]	train's l1: 0.0502579	val's l1: 0.0514883
[1405]	train's l1: 0.0502564	val's l1: 0.0514882
[1406]	train's l1: 0.0502551	val's l1: 0.0514882
[1407]	train's l1: 0.0502539	val's l1: 0.0514882
[1408]	train's l1: 0.0502526	val's l1: 0.0514882
[1409]	train's l1: 0.0502513	val's l1: 0.0514882
[1410]	train's l1: 0.0502502	val's l1: 0.0514883
[1411]	train's l1: 0.050249	val's l1: 0.0514882
[1412]	train's l1: 0.0502479	val's l1: 0.0514883
[1413]	train's l1: 0.0

In [ ]:
# Plot LightGBM feature importance
lgb.plot_importance(clf, height=0.8, figsize=(12.5, 12.5), ignore_zero=False)

In [10]:
%%time
def predict_and_export(clf, prop_2016, prop_2017, file_name):
    # Construct DataFrame for prediction results
    submission = pd.DataFrame()
    submission['ParcelId'] = prop_2016.parcelid
    
    # Construct features and make prediction
    test_features_2016 = data_proc.drop_features(prop_2016)
    test_features_2017 = data_proc.drop_features(prop_2017)
    
    test_dates = ['2016-10-01']
    test_columns = ['201610']
    for d, c in zip(test_dates, test_columns):
        print("Start predicting for {}".format(c))
        
        test_features = test_features_2016
        test_features['transactiondate'] = d
        test_features = data_proc.add_datetime_aggregate_features(test_features,
                                                                  logerror_year, logerror_month, logerror_quarter)

        pred_test = clf.predict(test_features)
        pred_test = [float(format(x, '.4f')) for x in pred_test]
        submission[c] = pred_test
        
        print("Finished predicting for {}".format(c))
    
    submission['201611'] = submission['201610']
    submission['201612'] = submission['201610']
    submission['201710'] = submission['201610']
    submission['201711'] = submission['201610']
    submission['201712'] = submission['201610']
    
    print("Length of submission DataFrame: {}".format(len(submission)))
    print("Submission header:")
    print(submission.head())
    submission.to_csv(file_name, index=False)
    return submission  # Return the results just in case we want to analyze or sanity check it

submission = predict_and_export(clf, prop_2016, prop_2017, 'data/test.csv')

Start predicting for 201610
Finished predicting for 201610
Length of submission DataFrame: 2985217
Submission header:
   ParcelId  201610  201611  201612  201710  201711  201712
0  10754147 -0.0226 -0.0226 -0.0226 -0.0226 -0.0226 -0.0226
1  10759547  0.0037  0.0037  0.0037  0.0037  0.0037  0.0037
2  10843547  0.0129  0.0129  0.0129  0.0129  0.0129  0.0129
3  10859147  0.0216  0.0216  0.0216  0.0216  0.0216  0.0216
4  10879947 -0.0023 -0.0023 -0.0023 -0.0023 -0.0023 -0.0023
CPU times: user 39min 51s, sys: 11.7 s, total: 40min 3s
Wall time: 10min 28s


In [ ]:
# Sweep a single hyperparameter
def sweep_hyperparam(param_name, sweep):
    feature_names = [s for s in lgb_features.columns]
    categorical_indices = []
    for i, t in enumerate(lgb_features.dtypes):
        if t.name == 'category':
            categorical_indices.append(i)

    lgb_train_set = lgb.Dataset(X_train, label=y_train, feature_name=feature_names)
    lgb_valid_set = lgb.Dataset(X_val, label=y_val, feature_name=feature_names)

    train_scores = []
    val_scores = []
    for p in sweep:
        print(p)
        np.random.seed(42)
        random.seed(36)
        params[param_name] = p
        clf = lgb.train(params, lgb_train_set, verbose_eval=False,
                    valid_sets=[lgb_train_set, lgb_valid_set], valid_names=['train', 'val'],
                    categorical_feature=categorical_indices)
        train_scores.append(abs(clf.predict(X_train) - y_train).mean() * 100)
        val_scores.append(abs(clf.predict(X_val) - y_val).mean() * 100)

    for i in range(len(sweep)):
        print("{}: train {}, val {}".format(sweep[i], train_scores[i], val_scores[i]))

sweep_hyperparam('learning_rate', [0.001, 0.0012, 0.0015, 0.0018, 0.002])